In [33]:
# Importations
from src.covid_cytof_dataset import CovidCytofDataset
from src.models.simple_nn import SimpleNeuralNetwork
from src.trainer import Trainer
import torch
from torch.utils.data import DataLoader, BatchSampler, random_split
from itertools import chain
from torch import nn

In [34]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [35]:
BATCH_SIZE = 10000

In [36]:
data = CovidCytofDataset("./data/attachments/COVID_CYTOF_BASIC_METADATA_STATUS_AGE_GROUP_SEX.xlsx", "./data", BATCH_SIZE)

/Users/gabriellechiron/Library/Caches/pypoetry/virtualenvs/covid-cytof-kKdDJC43-py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Loading fcs data:


100%|██████████| 99/99 [00:19<00:00,  5.12it/s]


done


In [37]:
train_data, test_data = random_split(list(BatchSampler(data, batch_size=BATCH_SIZE, drop_last=False)), [0.8, 0.2])
train_data, test_data = list(chain.from_iterable(train_data)), list(chain.from_iterable(test_data))

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

# del train_data, test_data

In [38]:
model = SimpleNeuralNetwork(data.data.shape[1]).to(device)
print(model)

SimpleNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=61, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [39]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

trainer = Trainer(model, optimizer, loss_fn, device, epochs=50)

In [40]:
trainer.run(train_dataloader, test_dataloader)

/Users/gabriellechiron/Library/Caches/pypoetry/virtualenvs/covid-cytof-kKdDJC43-py3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([10000])) that is different to the input size (torch.Size([10000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Train Epoch: 1 [0/800000 (0%)]	Loss: 0.315401


KeyboardInterrupt: 